In [ ]:
mkdir squad

mkdir: cannot create directory ‘squad’: File exists


In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2021-03-22 17:35:34--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M   212MB/s    in 0.2s    

2021-03-22 17:35:34 (212 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-03-22 17:35:34--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

squ

In [ ]:
import json
from pathlib import Path

def read_squad(path):
  path = Path(path)
  with open(path, 'rb') as f:
    squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts , questions ,answers

train_contexts,train_questions,train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json') 


In [ ]:
train_answers[1]

{'answer_start': 207, 'text': 'singing and dancing'}

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
train_answers[3]

{'answer_end': 180, 'answer_start': 166, 'text': 'Houston, Texas'}

In [ ]:
! pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_contexts ,train_questions ,truncation=True,padding =True)
val_encodings = tokenizer(val_contexts ,val_questions ,truncation=True,padding =True)

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)


In [ ]:
train_answers[3]

{'answer_end': 180, 'answer_start': 166, 'text': 'Houston, Texas'}

In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
from transformers import DistilBertForQuestionAnswering, Trainer, TrainingArguments
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,6.228300
20,6.202100
30,6.177700
40,6.130500
50,6.035200
60,5.932500
70,5.744200
80,5.429400
90,5.192900
100,4.871200


Step,Training Loss
10,6.228300
20,6.202100
30,6.177700
40,6.130500
50,6.035200
60,5.932500
70,5.744200
80,5.429400
90,5.192900
100,4.871200


TrainOutput(global_step=16281, training_loss=1.024831517843494, metrics={'train_runtime': 14593.6954, 'train_samples_per_second': 1.116, 'total_flos': 5.310098044580045e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 324322, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1453084, 'train_mem_gpu_alloc_delta': 797203456, 'train_mem_cpu_peaked_delta': 98017065, 'train_mem_gpu_peaked_delta': 6540899328})

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 1.3704547882080078,
 'eval_mem_cpu_alloc_delta': 72929,
 'eval_mem_cpu_peaked_delta': 188484,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 2315895808,
 'eval_runtime': 436.6396,
 'eval_samples_per_second': 46.496}

In [ ]:
test_context = """
Machine learning (ML) is the study of computer algorithms that improve automatically through experience. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.
"""
question = "What are machine learning models based on?"

In [ ]:
test_encoding = tokenizer(test_context, question, truncation=True, padding=True)

In [ ]:
test_encoding

{'input_ids': [101, 3698, 4083, 1006, 19875, 1007, 2003, 1996, 2817, 1997, 3274, 13792, 2008, 5335, 8073, 2083, 3325, 1012, 2009, 2003, 2464, 2004, 1037, 2112, 1997, 7976, 4454, 1012, 3698, 4083, 13792, 3857, 1037, 2944, 2241, 2006, 7099, 2951, 1010, 2124, 2004, 1000, 2731, 2951, 1000, 1010, 1999, 2344, 2000, 2191, 20932, 2030, 6567, 2302, 2108, 12045, 16984, 2000, 2079, 2061, 1012, 3698, 4083, 13792, 2024, 2109, 1999, 1037, 2898, 3528, 1997, 5097, 1010, 2107, 2004, 10373, 22910, 1998, 3274, 4432, 1010, 2073, 2009, 2003, 3697, 2030, 4895, 7959, 21369, 3468, 2000, 4503, 7511, 13792, 2000, 4685, 1996, 2734, 8518, 1012, 102, 2054, 2024, 3698, 4083, 4275, 2241, 2006, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)

In [ ]:
import numpy as np


input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)
with torch.no_grad():
    outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
start_label_indices = np.argmax(outputs[0].to('cpu').numpy())
end_label_indices = np.argmax(outputs[1].to('cpu').numpy())


In [ ]:

print(start_label_indices)
print(end_label_indices)

36
37


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
result =  " ".join(tokens[start_label_indices:end_label_indices+1]) 

In [ ]:
print(question)
print(f"Answer : {result}")

What are machine learning models based on?
Answer : sample data


In [ ]:
context = """The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) 
  were the people who in the 10th and 11th centuries gave their name to 
  Normandy, a region in France."""

question_1 = 'Where is Normandy'
question_2 = "What are Normans called in latin?"


In [ ]:
test_encoding_1 = tokenizer(context, question_1, truncation=True, padding=True)
test_encoding_2 = tokenizer(context, question_2, truncation=True, padding=True)

In [ ]:
def to_check_result(test_encoding):
    input_ids = torch.tensor(test_encoding["input_ids"]).to(device)
    attention_mask = torch.tensor(test_encoding["attention_mask"]).to(device)
    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask.unsqueeze(0))
    start_label_indices = np.argmax(outputs[0].to('cpu').numpy())
    end_label_indices = np.argmax(outputs[1].to('cpu').numpy())
    return start_label_indices, end_label_indices

In [ ]:
start_1, end_1 = to_check_result(test_encoding_1)
start_2, end_2 = to_check_result(test_encoding_2)

In [ ]:
tokens_1 = tokenizer.convert_ids_to_tokens(test_encoding_1["input_ids"])
tokens_2 = tokenizer.convert_ids_to_tokens(test_encoding_2["input_ids"])
result_1 =  " ".join(tokens_1[start_1:end_1 + 1])
result_2 =  " ".join(tokens_2[start_2:end_2 + 1]) 

In [ ]:
print(question_1)
print(f"Answer : {result_1}")

Where is Normandy
Answer : france


In [ ]:
print(question_2)
print(f"Answer : {result_2}")

What are Normans called in latin?
Answer : norman ##ni
